# Common Visualization Techniques in Pandas

Pandas is perhaps the most popular library in data science when it comes to analyzing data. It's power comes from pulling the entire dataset in the memory for fast access and creating DataFrames to provide a great way to visualize, slice and dice data. While this way of handling memory limits its appropriateness with very large datasets, pandas is the King when it comes to large datasets up until the mid-single digits in GB. Apart from being an analytical library, it also gives the user the ability to create quick visualizations using a easy to use syntax. True, it doesn't have the flexibility of a complete visualization library like Matplotlib or Seaborn, but this feature is incredibly useful in the earlier stages of analysis, prototyping and modeling. In this post we'll look at some of the more common visualization techniques in the way of exploring a dataset on New York City street tree census from 2015. Please note that this is by no means supposed to be an exploratory data analysis of the dataset. My intent in using this dataset is simpy to use it to demonstrate the various techniques. So let's get started without wasting any more digital ink.

As usual, we start by importing the Pandas library and loading the data...

```import pandas as pd
import matplotlib.pyplot as plt```

We are also going to import matplotlib to highlight some cases where Pandas fall short. As we shall see, that doesn't take away anything from the fact that pandas still provides enough visualization features for early stages of data science work.

```tree_data=pd.read_csv('2015_Street_Tree_Census_-_Tree_Data.csv')```

A quick look reveals that the newly created tree_data dataframe has 45 columns. For our purposes we select only 6 columns: 'tree_id', tree diameter at breast height ('tree_dbh'), whether tree is alive or a stump or dead ('status'),'health' and species name ('spc_latin'). We save this DataFrame as **dbhStatusHealthSpecies**, ready for futher analysis and visualization.

It's good to be cognizant of how plotting methods work in pandas. As mentioned in the official [documentation](https://pandas.pydata.org/pandas-docs/version/0.13/visualization.html), these methods are essentially wrappers around matplotlib. That is to say that under the hood, such methods send requests to matplotlib which renders the charts. This lends a lot of similarities in the syntax of how arguments are passed to matplotlib and various pandas methods. This will be evident starting right from the first type of chart we discuss - bar charts. 

## Bar Charts:

Let's say we want to visualize the top 10 most common species of trees in the census. We can do this using the new **dbhStatusHealthSpecies** dataframe by using the following syntax:

```dbhStatusHealthSpecies[['spc_latin']].value_counts().head(10).plot.bar(xlabel='tree species(latin name)',ylabel='count',alpha=0.8,fontsize=12,figsize=(12,8))```

![bar chart without rcParams](vbar_without_rcParams.png)

That looks long and cryptic! But wait, once we break it down it won't be as intimidating. The first part:

```dbhStatusHealthSpecies[['spc_latin']].value_counts().head(10)```

uses pandas methods ```value_counts()``` to create a table showing the number of trees for each species. The ```head(10)``` is added to select the top 10 most common species. DataFrames in pandas have a ```plot``` method and the next part:  

```plot.bar(xlabel='tree species(latin name)',ylabel='count',alpha=0.8,fontsize=12,figsize=(12,8))```

uses that to plot a ```.bar``` chart. We specify names of labels for x and y axes and pass common attributes such as ```alpha```, ```fontsize``` and ```figsize``` in parentheses. Notice the tiny font size for the axis labels, however. Unfortunately, not everything can be tuned from inside a DataFrame plot method. Size of axis labels is one such thing. Instead, we can set it by dynamically changing matplotlib configuration. For example:  

```plt.rcParams.update({'axes.labelsize': 18})```  

results in the chart below. Particularly in the analytical stage, however, this hardly takes anything awayfrom the usefulness of pandas plotting capabilities.

![bar chart with rcParams](vbar_with_rcParams.png)

Oftentimes a horizontal bar chart is easier to read than a vertical bar chart. Such a chart can be created much in the same way:

```pd.DataFrame(dbhStatusHealthSpecies[['spc_latin']].value_counts()).sort_values(by=[0]).head(10).plot.barh(xlabel='count',ylabel='tree species(latin name)',alpha=0.8,fontsize=12,figsize=(12,8)).get_legend().remove()```


```plt.rcParams.update({'axes.labelsize': 20})```

![horizontal bar](hbar_with_rcParams.png)

## Histogram

Histograms are a great way to visualize distribution of a variable. Thankfully, DataFrame objects in pandas have a method to create histograms, appropriately named ```hist()```. A histogram can be created by calling this method for a particular column in the DataFrame. As shown in the example below, there are quite a few chart attributes that can be set from inside ```hist()```. A complete list of such attributes can be found in the pandas [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.hist.html#pandas.DataFrame.hist).  

```ax=dbhStatusHealthSpecies['status'].hist(bins=range(4),align='mid',grid=False,color='teal',alpha=0.8,figsize=(6,6));```  
```ax.tick_params(labelsize=13)
ax.set_xlabel('Tree Status',fontsize=14,loc='center')
ax.set_ylabel('Counts', fontsize=14)```

The ```ax```'s may appear confusing at first. Similar to the```plot``` method, not every attribute of a chart can be accessed from within ```hist```. Labeling axes and sizing tick parameters are just a few such attributes. However just like everything else in python, the histogram is an object. This allows us to set such attributes quite easily by passing them to matplotlib under the hood. That is exactly what we are doing in the example here. We name the object ```ax``` and pass desired values for attributes.

![histogram](histogram.png)

## Boxplot

The final type of visualization we'll look at is boxplot. They provide data scientists a way to visualize several aspects of data in a very compact way. Here is a figure with details of all the features of a boxplot:

![interpreting box plots](reading_boxplots.png)

(Source: [Understanding Boxplots](https://towardsdatascience.com/understanding-boxplots-5e2df7bcbd51), Michael Galarnyk, Towards Datascience)

As usual a data scientist always needs to keep their audience in mind. Boxplots are a very useful tool in day to day work of a data scientist. They can also be a great option while presenting results to a technical audience. But there is a caveat. Their complex nature make them too difficult for a nontechnical audience. 

Now let's take a look at how to create a boxplot in pandas. It is often helpful to create a new DataFrame with only those types that we want to analyze using boxplots. Let's say we want to plot boxplots of 'diameter at breast height' (tree_dbh) of two species: 'Platanus x acerifolia' and 'Styphnolobium japonicum'. We start by creating a new DataFrame df with data on only those two species. Next, to clear up any ambiguities in instruction given to pandas, we remove all the other numeric columns. In our example this is the column 'tree id'.

```df=dbhStatusHealthSpecies[(dbhStatusHealthSpecies['spc_latin']=='Platanus x acerifolia')|(dbhStatusHealthSpecies['spc_latin']=='Styphnolobium japonicum')]```

```df.drop('tree_id',axis=1,inplace=True)```

In the final step we group 'tree_dbh' by species and use the ```boxplot``` method of dataframes to generate the boxplots. Pandas [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.boxplot.html) has a complete list of attributes that could be set from within ```boxplot```. All plotting keyword arguments are passed to ```pyplot.boxplot()``` of matplotlib.

```df.groupby('spc_latin')[['tree_dbh']].boxplot(figsize=(10,8));```

![boxplot](boxplot.png)